# Multiclass Classification of White Wine Quality with Support Vector Machine and Feature Union

## Wine Data
Data from http://archive.ics.uci.edu/ml/datasets/Wine+Quality

### Citations
<pre>
Dua, D. and Karra Taniskidou, E. (2017). 
UCI Machine Learning Repository [http://archive.ics.uci.edu/ml/index.php]. 
Irvine, CA: University of California, School of Information and Computer Science.
</pre>

<pre>
P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis. 
Modeling wine preferences by data mining from physicochemical properties.
In Decision Support Systems, Elsevier, 47(4):547-553. ISSN: 0167-9236.
</pre>

Available at:
- [@Elsevier](http://dx.doi.org/10.1016/j.dss.2009.05.016)
- [Pre-press (pdf)](http://www3.dsi.uminho.pt/pcortez/winequality09.pdf)
- [bib](http://www3.dsi.uminho.pt/pcortez/dss09.bib)

## Setup

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

white_wine = pd.read_csv('../../ch_10/data/winequality-white.csv', sep=';')

## EDA

In [ ]:
white_wine.head()

In [ ]:
white_wine.describe()

In [ ]:
white_wine.info()

In [ ]:
def plot_quality_scores(df, kind):
    ax = df.quality.value_counts().sort_index().plot.barh(
        title=f'{kind.title()} Wine Quality Scores', figsize=(12, 3)
    )
    ax.axes.invert_yaxis()
    for bar in ax.patches:
        ax.text(
            bar.get_width(), 
            bar.get_y() + bar.get_height()/2, 
            f'{bar.get_width()/df.shape[0]:.1%}',
            verticalalignment='center'
        )
    plt.xlabel('count of wines')
    plt.ylabel('quality score')

    for spine in ['top', 'right']:
        ax.spines[spine].set_visible(False)

    return ax

plot_quality_scores(white_wine, 'white')

## White wine quality multiclass classification

### Train test split

In [ ]:
from sklearn.model_selection import train_test_split

y = white_wine.quality
X = white_wine.drop(columns=['quality'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.15, random_state=0, stratify=y
)

### Build model

In [ ]:
%%capture --no-stdout
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFpr
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.svm import SVC

pipeline = Pipeline([
    ('scale', StandardScaler()), 
    ('feature_union', FeatureUnion([
        ('interaction_terms', PolynomialFeatures(degree=2, interaction_only=True)), 
        ('fpr', SelectFpr())
    ])),
    ('svm', SVC(gamma='auto', random_state=0, probability=True))
])

search_space = {
    'feature_union__interaction_terms__include_bias': [True, False],
    'svm__C': [0.5, 1, 5]
}

grid = GridSearchCV(pipeline, search_space, scoring='f1_macro', cv=5).fit(X_train, y_train)

Check the best hyperparameters:

In [ ]:
grid.best_params_

### Evaluating the model
Get predictions:

In [ ]:
preds = grid.predict(X_test)

Look at the classification report:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, preds))

Review the confusion matrix:

In [ ]:
from ml_utils.classification import confusion_matrix_visual

confusion_matrix_visual(y_test, preds, np.sort(white_wine.quality.unique()))

Look at the precision-recall curves:

In [ ]:
from ml_utils.classification import plot_multiclass_pr_curve
plot_multiclass_pr_curve(y_test, grid.predict_proba(X_test))

<hr>
<div>
    <a href="./exercise_1.ipynb">
        <button>&#8592; Previous Solution</button>
    </a>
    <a href="./exercise_3.ipynb">
        <button style="float: right;">Next Solution &#8594;</button>
    </a>
</div>
<hr>